In [99]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone

#### Trial

In [100]:
s1_acc = pd.read_csv('data/S1/Final/ACC.csv', header=None)
s1_acc.columns = ['x', 'y', 'z']
s1_acc

,x,y,z
0,1.544027e+09,1.544027e+09,1.544027e+09
1,3.200000e+01,3.200000e+01,3.200000e+01
2,-3.000000e+00,6.500000e+01,6.000000e+00
3,-3.000000e+00,6.500000e+01,6.000000e+00
4,-3.000000e+00,6.500000e+01,6.000000e+00
...,...,...,...
748683,-2.400000e+01,4.200000e+01,4.200000e+01
748684,-2.400000e+01,4.200000e+01,4.300000e+01
748685,-2.500000e+01,4.200000e+01,4.200000e+01
748686,-2.400000e+01,4.200000e+01,4.400000e+01


In [101]:
timestamp = s1_acc.iloc[0, 0]  # Given Unix timestamp
utc_time = datetime.fromtimestamp(timestamp, tz=timezone.utc)
utc_time

datetime.datetime(2018, 12, 5, 16, 28, 57, tzinfo=datetime.timezone.utc)

In [102]:
session_start_unix = s1_acc.iloc[0, 0]
sample_rate = s1_acc.iloc[1, 0]
print("Sample Rate:", sample_rate)

session_start_dt = datetime.fromtimestamp(session_start_unix, tz=timezone.utc)
exam_start_dt = session_start_dt.replace(hour=session_start_dt.hour+1, minute=0, second=0, microsecond=0)
print("Session Start Datetime:", session_start_dt)
print("Exam Start Datetime:", exam_start_dt)


# Convert exam start time to Unix timestamp
exam_start_time = int(exam_start_dt.timestamp())

# Define other key timestamps
halfway_time = exam_start_time + (10_800 / 2)  # Halfway (1.5 hours after exam start)
exam_end_time = exam_start_time + 10_800  # Exam end (3 hours after exam start)

# Compute actual timestamps for each row
s1_acc = s1_acc.iloc[2:].reset_index(drop=True)  # Remove first two rows (start time & sample rate)
s1_acc['Timestamp'] = session_start_unix + (s1_acc.index / sample_rate)  # Convert row index to actual timestamps


conditions = [
    (s1_acc['Timestamp'] < exam_start_time),
    (s1_acc['Timestamp'] >= exam_start_time) & (s1_acc['Timestamp'] < halfway_time),
    (s1_acc['Timestamp'] >= halfway_time) & (s1_acc['Timestamp'] < exam_end_time),
    (s1_acc['Timestamp'] >= exam_end_time),
]

labels = ['0-Before Exam', '1-Exam Start to Halfway', '2-Halfway to Exam End', '3-After Exam']

# Assign segment labels
s1_acc['Segment'] = np.select(conditions, labels)

s1_acc

Sample Rate: 32.0
Session Start Datetime: 2018-12-05 16:28:57+00:00
Exam Start Datetime: 2018-12-05 17:00:00+00:00


,x,y,z,Timestamp,Segment
0,-3.0,65.0,6.0,1.544027e+09,0-Before Exam
1,-3.0,65.0,6.0,1.544027e+09,0-Before Exam
2,-3.0,65.0,6.0,1.544027e+09,0-Before Exam
3,-3.0,65.0,6.0,1.544027e+09,0-Before Exam
4,-3.0,65.0,6.0,1.544027e+09,0-Before Exam
...,...,...,...,...,...
748681,-24.0,42.0,42.0,1.544051e+09,3-After Exam
748682,-24.0,42.0,43.0,1.544051e+09,3-After Exam
748683,-25.0,42.0,42.0,1.544051e+09,3-After Exam
748684,-24.0,42.0,44.0,1.544051e+09,3-After Exam


In [105]:
conver = 9.81 / 64 # convert to m/s^2
s1_acc['A_total'] = np.sqrt(s1_acc.iloc[:, 0]**2 + s1_acc.iloc[:, 1]**2 + s1_acc.iloc[:, 2]**2) * conver
s1_acc.groupby('Segment', observed=True)['A_total'].mean()

Segment
0-Before Exam              9.968975
1-Exam Start to Halfway    9.930741
2-Halfway to Exam End      9.952267
3-After Exam               9.657924
Name: A_total, dtype: float64

#### Func

In [110]:
def get_data(sid, exam, time_adjustment=1):
    acc = pd.read_csv(f'data/{sid}/{exam}/ACC.csv', header=None)
    acc.columns = ['x', 'y', 'z']
    
    session_start_unix = acc.iloc[0, 0]
    sample_rate = acc.iloc[1, 0]
    
    session_start_dt = datetime.fromtimestamp(session_start_unix, tz=timezone.utc)
    exam_start_dt = session_start_dt.replace(hour=session_start_dt.hour+time_adjustment, minute=0, second=0, microsecond=0)
    exam_start_time = int(exam_start_dt.timestamp())
    if exam == 'Final':
        halfway_time = exam_start_time + (10_800 / 2)  # Halfway (1.5 hours after exam start)
        exam_end_time = exam_start_time + 10_800  # Exam end (3 hours after exam start)
    else:
        halfway_time = exam_start_time + (10_800 / 4)  # Halfway (1.5 hours after exam start)
        exam_end_time = exam_start_time + (10_800 / 2)  # Exam end (3 hours after exam start)
    
    acc = acc.iloc[2:].reset_index(drop=True)  # Remove first two rows (start time & sample rate)
    acc['Timestamp'] = session_start_unix + (acc.index / sample_rate)
    
    conditions = [
    (acc['Timestamp'] < exam_start_time),
    (acc['Timestamp'] >= exam_start_time) & (acc['Timestamp'] < halfway_time),
    (acc['Timestamp'] >= halfway_time) & (acc['Timestamp'] < exam_end_time),
    (acc['Timestamp'] >= exam_end_time),
    ]

    labels = ['0-Before Exam', '1-Exam Start to Halfway', '2-Halfway to Exam End', '3-After Exam']

    acc['Segment'] = np.select(conditions, labels)
    
    conver = 9.81 / 64 # convert to m/s^2
    acc[f'{sid}_{exam}'] = np.sqrt(acc.iloc[:, 0]**2 + acc.iloc[:, 1]**2 + acc.iloc[:, 2]**2) * conver
    return acc.groupby('Segment')[f'{sid}_{exam}'].mean().reset_index()
    

In [115]:
def time_check(sid, exam):
    acc = pd.read_csv(f'data/{sid}/{exam}/ACC.csv', header=None)
    acc.columns = ['x', 'y', 'z'] 
    print(datetime.fromtimestamp(acc.iloc[0, 0], tz=timezone.utc))

In [116]:
all_data = [(sid, exam) for sid in [f"S{i}" for i in range(1, 11)] for exam in ['Final', 'Midterm 1', 'Midterm 2']]
for sid, exam in all_data:
    time_check(sid, exam)

2018-12-05 16:28:57+00:00
2018-10-13 12:56:06+00:00
2018-11-10 14:19:15+00:00
2018-12-05 16:29:04+00:00
2018-10-13 12:55:45+00:00
2018-11-10 14:17:29+00:00
2018-12-05 16:28:54+00:00
2018-10-13 12:56:48+00:00
2018-11-10 14:16:34+00:00
2018-12-05 16:29:02+00:00
2018-10-13 12:56:19+00:00
2018-11-10 14:18:53+00:00
2018-12-05 16:28:54+00:00
2018-10-13 12:55:21+00:00
2018-11-10 14:18:18+00:00
2018-12-05 16:29:07+00:00
2018-10-13 12:57:13+00:00
2018-11-10 14:18:08+00:00
2018-12-05 16:28:56+00:00
2018-10-13 12:56:47+00:00
2018-11-10 14:16:43+00:00
2018-12-05 16:28:59+00:00
2018-10-13 12:56:20+00:00
2018-11-10 14:16:24+00:00
2018-12-05 16:29:07+00:00
2018-10-13 12:56:05+00:00
2018-11-10 14:17:35+00:00
2018-12-05 16:28:54+00:00
2018-10-13 12:55:25+00:00
2018-11-10 14:18:45+00:00


In [121]:
index = 0
for sid, exam in all_data:
    if index == 0:
        all_exam_data = get_data(sid, exam)
        index += 1
    else:
        all_exam_data = all_exam_data.merge(get_data(sid, exam), on='Segment')
all_exam_data

,Segment,S1_Final,S1_Midterm 1,S1_Midterm 2,S2_Final,S2_Midterm 1,S2_Midterm 2,S3_Final,S3_Midterm 1,S3_Midterm 2,...,S7_Midterm 2,S8_Final,S8_Midterm 1,S8_Midterm 2,S9_Final,S9_Midterm 1,S9_Midterm 2,S10_Final,S10_Midterm 1,S10_Midterm 2
0,0-Before Exam,9.968975,9.689174,9.820555,9.901920,9.714896,9.728224,9.954444,9.701667,10.130541,...,9.683050,10.078016,9.662011,9.865701,9.933465,9.732078,9.933655,9.887947,9.831823,9.886236
1,1-Exam Start to Halfway,9.930741,9.913496,9.921388,9.799262,9.772192,9.484924,10.009542,9.763239,10.016960,...,9.806775,10.025887,9.821738,9.971504,10.105543,9.763360,10.083786,9.866637,9.860932,9.880708
2,2-Halfway to Exam End,9.952267,9.997782,9.922328,9.790479,9.584494,9.459354,10.048323,10.097318,10.004672,...,9.802444,9.990508,9.967601,9.995136,10.093374,10.121194,10.088068,9.873721,9.884537,9.882101
3,3-After Exam,9.657924,9.912415,9.942897,9.876522,9.577238,9.760280,9.756930,9.855780,9.975387,...,9.818746,9.943809,9.946189,9.999127,9.746913,10.025526,10.017029,9.743140,9.873406,9.958041


In [124]:
grade = pd.read_csv('data/ParsedGrades.csv')
grade

,Student,Exam,Score
0,S1,Midterm 1,78
1,S2,Midterm 1,82
2,S3,Midterm 1,77
3,S4,Midterm 1,75
4,S5,Midterm 1,67
5,S6,Midterm 1,71
6,S7,Midterm 1,64
7,S8,Midterm 1,92
8,S9,Midterm 1,80
9,S10,Midterm 1,89
